In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from nltk.stem.snowball import SnowballStemmer 
from sklearn.compose import ColumnTransformer
import nltk

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     E:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     E:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
data = pd.read_csv('data/data.csv', delimiter=';', index_col='ID')
train = pd.read_csv('data/train.csv', delimiter=';')
test = pd.read_csv('data/test.csv', delimiter=';')

In [4]:
print(f"Количество вопросов {data.shape[0]}. Из них в обучающей выборке {train.shape[0]} ,"
      f" в тестовой -- {test.shape[0]}")

Количество вопросов 41087. Из них в обучающей выборке 30000 , в тестовой -- 11087


In [5]:
X_train = data.loc[train.ID]
y_train = train.set_index('ID')
X_test = data.loc[test.ID]

In [6]:
def random_pick_n(iterable, n, random_seed=43):
    np.random.seed(random_seed)
    if type(iterable) is pd.DataFrame:
        ids = np.random.choice(len(iterable), n, replace=False)
        return iterable.iloc[ids]
    if type(iterable) is np.ndarray and len(iterable.shape) > 1:
        return 0
    return np.random.choice(iterable, n, replace=False)

In [7]:
for s in random_pick_n(X_train[y_train.Answer == 0], 20, random_seed=137).values:
    print(s)

['Что из этого не является морем ВАРИАНТЫ ОТВЕТА КАСПИСКОЕ ЧЁРНОЕ БЕРИНГОВО']
['В музее какого писателя , есть зал, в котором хранятся 74 топора']
['Какая телепередача являлась самой популярной на 2015']
['Сколько кампаний  Avon проходит за год']
['У кого из этих футболистов больше всего золотых мячей?']
['в каком из этих городов проживает меньше всего людей?']
['Какой из этих городов, НЕ находится в Германии?']
['Мой друг вывел деньги на виртуальную карту QIWI 2O9 рублей,они таки не пришли,что делать?Номер карты 4890 4944 9922 4699']
['Высота небоскреба «Бурж Халифа»']
['Какой из названных автомобилей самый быстрый?']
['С помощью какого апаората можно оставаться на связи с друзьями и близкими?']
['Откуда родом Мухаммед Али']
['В каком году появилось первое мороженое?']
['Сколько полей-квадратиков на шахматной доске?']
['В каком году умер Александр 2?']
['Сколько раз старик из сказки А. С. Пушкина вызывал Золотую рыбку?']
['Как звали жениха Полли Купер в сериале Ривердейл?']
['Негласны

In [8]:
for s in random_pick_n(X_train[y_train.Answer == 1], 20, random_seed=137).values:
    print(s)

['Что такое «рамбутан»?']
['В каком году была битва за Брестскую крепость?']
['Как звучало любимое заклинание героя мультфильма «Чудовище»?']
['Кто стал победителем в беге на 100 метров на Олимпийских играх 1936 года в Берлине?']
['Что изучает ботаника?']
['Кто считается «отцом теории информации»?']
['Когда родилась Алла Пугачёва?']
['Когда Западная Римская империя прекратила своё существование?']
['Какая  столица у Намибии?']
['В какой стране носят шапки из своих волос?']
['Кто самый первый получил Нобелевскую премию по физиологии или медицине?']
['Как зовут создателя паровой машины?']
['Какой танк участвовал в Первой мировой войне?']
['Какой язык программирования существует?']
['Сколько просветов было на погонах есаула?']
['Какой футбольный клуб НЕ выигрывал Лигу Европы УЕФА?']
['Сколько протонов содержится в ядре одного атома криптона?']
['Какого города НЕТ в Швейцарии?']
['Кто из НЕ относится к Отцам-основателям США?']
['Кто сменил Ф. Э. Дзержинского после его смерти на посту главы

In [9]:
class CustomTfIdf(BaseEstimator, TransformerMixin):
    def __init__(self, min_df=1, max_df=1.0, preprocessor=None, analyzer='word', ngram_range=(1, 1)):
        self.min_df = min_df
        self.max_df = max_df
        self.preprocessor = preprocessor
        self.analyzer = analyzer
        self.ngram_range = ngram_range
    
    def fit(self, X, y=None):
        if X.shape[1] > 1:
            print(f"Внимание! X.shape = {X.shape} in CustomTfIdf.fit")
        self.vect = TfidfVectorizer(token_pattern=r'(?u)\b[А-я]+\b', min_df=self.min_df,
                                    max_df=self.max_df, preprocessor=self.preprocessor, norm=False,
                                    analyzer=self.analyzer, ngram_range=self.ngram_range)
        self.vect.fit(X.iloc[:, 0], y)
        return self
    
    def transform(self, X):
        if X.shape[1] > 1:
            print(f"Внимание! X.shape = {X.shape} in CustomTfIdf.transform")
        return self.vect.transform(X.iloc[:, 0])


class CustomFeatures(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        last_quest_mark = X['Question'].apply(lambda s: s[-1] == '?')
        first_capital = X['Question'].apply(lambda s: s[0].isupper())
        return pd.DataFrame({'lastQuestMark': last_quest_mark, 'firstCapital': first_capital})


def add_features_by_regexp(X, regexp, suffix):
    X['len_mean_'+suffix] = X['Question'].apply(lambda test_str: 0 if len(re.findall(regexp, test_str))==0 else np.mean([len(s) for s in re.findall(regexp, test_str)]))
    X['count_'+suffix] = X['Question'].apply(lambda test_str: len([len(s) for s in re.findall(regexp, test_str)]))
    return X

def add_independent_features(X, inplace=False):
    if inplace:
        X1 = X
    else:
        X1 = X.copy()
    X1['lastQuestMark'] = X1['Question'].apply(lambda s: s[-1] == '?')
    X1['firstCapital'] = X1['Question'].apply(lambda s: s[0].isupper())
    X1['HasCapitalNe'] = X1['Question'].apply(lambda s: (re.search(r'\bНЕ\b', s) is not None) or 
                       (re.search(r'\bНЕТ\b', s) is not None))
    X1['HasLowerNe'] = X1['Question'].apply(lambda s: (re.search(r'\bне\b', s) is not None) or 
                       (re.search(r'\bнет\b', s) is not None))
    X1['HasTwoSpaces'] = X1['Question'].apply(lambda s: re.search(r'\s\s', s) is not None)
    X1['HasSpaceBeforeDot'] = X1['Question'].apply(lambda s: re.search(r'\s[,.;:!?]', s) is not None)
    X1['HasStrangeStart'] = X1['Question'].apply(lambda s: re.search(r'^\W', s) is not None)
    
    X1 = add_features_by_regexp(X1, r"\b[А-Я]+\b", 'ru_caps')
    X1 = add_features_by_regexp(X1, r"\b[А-Яа-я]+\b", 'ru_all')
    X1 = add_features_by_regexp(X1, r"\b[А-Я][а-я]*\b", 'ru_names')
    X1 = add_features_by_regexp(X1, r"\b[A-Z]+\b", 'en_caps')
    X1 = add_features_by_regexp(X1, r"\b[A-Za-z]+\b", 'en_all')
    X1 = add_features_by_regexp(X1, r"\b[A-Z][a-z]*\b", 'en_names')
    X1 = add_features_by_regexp(X1, r"\b[0-9]+\b", 'numbers')
    
    special_chars = ['«', '»', '-', '"', '!', '—', ')', '(', '–', '+', '*', ':','\\', '/', '<', '>', '“', '”'
                 "'", '’', '=', '…', '„', '\xad', '\xa0', '_', '|']
    vect_spec_chars = TfidfVectorizer(analyzer='char', vocabulary=special_chars, use_idf=False, norm=False)
    X_spec_chars = vect_spec_chars.fit_transform(X['Question'])
    X_spec_chars = pd.DataFrame(X_spec_chars.toarray(), columns=special_chars, index=X.index)
    X1 = X1.join(X_spec_chars, rsuffix='_special_chars')
    
    return X1

In [10]:
%%time

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=43)
stemmer = SnowballStemmer("russian") 

def stem(sentence):
    words = nltk.word_tokenize(sentence, 'russian')
    words = [stemmer.stem(word) for word in words]
    return ' '.join(words)

tfidf = CustomTfIdf()
charTfidf = CustomTfIdf(analyzer='char', ngram_range=(2, 2))
clf = LogisticRegression()
clf = lgb.LGBMClassifier(n_estimators=50)

tfidfPipe = Pipeline([('tfidf', tfidf)]) 
charTfidfPipe = Pipeline([('charTfidf', charTfidf)])
preprocessor = ColumnTransformer(transformers=[('tfidfPipe', tfidfPipe, ['Question']),
                                               ('charTfidfPipe', charTfidfPipe, ['Question'])], 
                                 remainder='passthrough')

pipe = Pipeline([('preproc', preprocessor), ('clf', clf)])

params = {'preproc__tfidfPipe__tfidf__min_df': [1, 0.01, 0.1], 
          'preproc__tfidfPipe__tfidf__max_df': [0.4, 0.6, 0.9], 
          'clf__n_estimators': [10, 50, 100], 
          'preproc__tfidfPipe__tfidf__preprocessor': [stem]}
grid = GridSearchCV(pipe, params, scoring='roc_auc', cv=cv, n_jobs=-1)

grid.fit(X=add_independent_features(X_train), y=y_train['Answer'])

print(grid.best_score_)
print(grid.best_params_)


0.808559048882347
{'clf__n_estimators': 50, 'preproc__tfidfPipe__tfidf__max_df': 0.6, 'preproc__tfidfPipe__tfidf__min_df': 1, 'preproc__tfidfPipe__tfidf__preprocessor': <function stem at 0x0000029F2D1B3AF8>}
Wall time: 14min 48s


In [11]:
stemmer = SnowballStemmer("russian") 

def stem(sentence):
    words = nltk.word_tokenize(sentence, 'russian')
    words = [stemmer.stem(word) for word in words]
    return ' '.join(words)

tfidf = CustomTfIdf()
clf = lgb.LGBMClassifier(n_estimators=50)

tfidfPipe = Pipeline([('tfidf', tfidf)]) 
preprocessor = ColumnTransformer(transformers=[('tfidfPipe', tfidfPipe, ['Question'])], 
                                 remainder='passthrough')

pipe = Pipeline([('preproc', preprocessor), ('clf', clf)])

params = {'preproc__tfidfPipe__tfidf__min_df': 1, 'preproc__tfidfPipe__tfidf__max_df': 0.6, 
          'preproc__tfidfPipe__tfidf__preprocessor': stem}

pipe.set_params(**params)

pipe.fit(add_independent_features(X_train), y_train['Answer'])

y_test = pipe.predict_proba(add_independent_features(X_test))

In [12]:
pd.Series(y_test[:, 1], index=X_test.index).to_csv('submission.csv')




E:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
